# Model 2

In [2]:
import os
import torch
import random
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import defaultdict
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import DataLoader, Subset, random_split, Dataset

## initiation:

In [3]:
class NumpyFolderDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Dataset to handle .npy files organized by class in subdirectories.

        Args:
        - root_dir (str): Root directory containing class subdirectories.
        - transform (callable, optional): Transform to apply to each sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []  # List to store file paths and their labels

        # Traverse through the directory structure
        for class_idx, class_name in enumerate(sorted(os.listdir(root_dir))):
            class_dir = os.path.join(root_dir, class_name)
            if os.path.isdir(class_dir):
                for file_name in os.listdir(class_dir):
                    if file_name.endswith('.npy'):
                        file_path = os.path.join(class_dir, file_name)
                        self.samples.append((file_path, class_idx))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, label = self.samples[idx]
        np_array = np.load(file_path).astype(np.float32)  # Load the .npy file

        if self.transform:
            np_array = self.transform(np_array)
        return np_array, label

# Define transformations
transform = transforms.Compose([
    transforms.Lambda(lambda x: torch.tensor(x)),  # Convert NumPy array to PyTorch tensor
    transforms.Lambda(lambda x: x.unsqueeze(0)),   # Add channel dimension for CNN
])

# Path to your dataset
dataset_path = r'C:\Users\Admin\PycharmProjects\Ramin_Thesis\DataSet\12_class'

# Create the dataset
dataset = NumpyFolderDataset(root_dir=dataset_path, transform=transform)

# Split into training and test datasets
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Verify DataLoader
for batch_data, batch_labels in train_loader:
    print(f"Batch data shape: {batch_data.shape}, Batch labels: {batch_labels.shape}")
    break

Batch data shape: torch.Size([32, 1, 320, 320]), Batch labels: torch.Size([32])


### 1

In [5]:
class ImprovedCNNWithTransformer(nn.Module):
    def __init__(self, num_classes=12, num_transformer_layers=4, num_heads=8):
        super().__init__()

        # Convolutional Layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)

        self.shortcut = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=1, stride=2),
            nn.BatchNorm2d(128)
        )

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)

        # Transformer Parameters
        self.embed_dim = 128  # Token embedding size
        self.global_pool = nn.AdaptiveAvgPool2d((4, 4))  # Fixed-size output for tokenization

        # Transformer Encoder
        encoder_layer = TransformerEncoderLayer(d_model=self.embed_dim, nhead=num_heads, dim_feedforward=512, dropout=0.4)
        self.transformer = TransformerEncoder(encoder_layer, num_layers=num_transformer_layers)

        # Fully Connected Layers
        self.fc1 = nn.Linear(self.embed_dim, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        # Block 1
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)

        # Block 2
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)

        # Block 3 with Residual
        shortcut = self.shortcut(x)  # Downsample shortcut
        x = F.relu(self.bn3(self.conv3(x)) + shortcut)

        # Global Pooling
        x = self.global_pool(x)  # Shape: [batch_size, 128, 4, 4]
        batch_size, channels, height, width = x.size()

        # Prepare Transformer Input
        x = x.view(batch_size, channels, -1).permute(0, 2, 1)  # Shape: [batch, 16, 128]

        # Transformer Encoder
        x = self.transformer(x)  # Shape: [batch, 16, 128]
        x = x.mean(dim=1)  # Aggregate token representations (Shape: [batch, 128])

        # Fully Connected Layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x
model = ImprovedCNNWithTransformer()

## Eval Section:

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-2)
num_epochs = 200  # Assuming a total of 50 epochs
model.to(device)

# Variable to track the best model
best_test_accuracy = 0.0
best_model_path = "best_model.pth"

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # Training loop
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    # Calculate training metrics
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_samples
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, "
          f"Accuracy: {epoch_accuracy:.2%}, "
          f"LR: {optimizer.param_groups[0]['lr']:.6f}")

    # Start testing after 5th epoch
    if epoch >= 3:
        model.eval()
        correct_predictions = 0
        total_samples = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                correct_predictions += (predicted == labels).sum().item()
                total_samples += labels.size(0)

        # Calculate test accuracy
        test_accuracy = correct_predictions / total_samples
        print(f"Test Accuracy after Epoch {epoch+1}: {test_accuracy:.2%}")

        # Save the model if it's the best so far
        if test_accuracy > best_test_accuracy:
            best_test_accuracy = test_accuracy
            torch.save(model.state_dict(), best_model_path)
            print(f"Best model saved with accuracy: {best_test_accuracy:.2%}")

# Load the best model for further use
print(f"Training complete. Best model accuracy: {best_test_accuracy:.2%}")
model.load_state_dict(torch.load(best_model_path))


Epoch [1/30], Loss: 1.7978, Accuracy: 29.72%, LR: 0.000300
Epoch [2/30], Loss: 1.3190, Accuracy: 46.23%, LR: 0.000300
Epoch [3/30], Loss: 1.2457, Accuracy: 50.44%, LR: 0.000300
Epoch [4/30], Loss: 1.0384, Accuracy: 58.00%, LR: 0.000300
Test Accuracy after Epoch 4: 61.00%
Best model saved with accuracy: 61.00%
Epoch [5/30], Loss: 0.9606, Accuracy: 60.54%, LR: 0.000300
Test Accuracy after Epoch 5: 39.10%
Epoch [6/30], Loss: 0.9104, Accuracy: 62.83%, LR: 0.000300
Test Accuracy after Epoch 6: 60.90%
Epoch [7/30], Loss: 0.8618, Accuracy: 64.14%, LR: 0.000300
Test Accuracy after Epoch 7: 56.09%
Epoch [8/30], Loss: 0.8200, Accuracy: 66.01%, LR: 0.000300
Test Accuracy after Epoch 8: 66.24%
Best model saved with accuracy: 66.24%
Epoch [9/30], Loss: 0.7938, Accuracy: 66.81%, LR: 0.000300
Test Accuracy after Epoch 9: 67.31%
Best model saved with accuracy: 67.31%
Epoch [10/30], Loss: 0.7500, Accuracy: 68.59%, LR: 0.000300
Test Accuracy after Epoch 10: 68.80%
Best model saved with accuracy: 68.80%


C:\Users\Admin\AppData\Local\Temp\ipykernel_12068\2136515234.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


<All keys matched successfully>

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm

# Define your class names manually
class_names = ["Class_1", "Class_2", "Class_3", "Class_4",
               "Class_5", "Class_6", "Class_7", "Class_8",
               "Class_9", "Class_10", "Class_11", "Class_12"]  # Replace with actual class names

def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            # Get predictions
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Convert to numpy arrays for evaluation
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Overall accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Overall Test Accuracy: {accuracy:.4f}")

    # Per-class accuracy
    class_accuracy = np.diag(confusion_matrix(all_labels, all_preds)) / np.bincount(all_labels)
    for i, acc in enumerate(class_accuracy):
        print(f"Class {i} Accuracy: {acc:.4f}")

    # Classification report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    # Confusion matrix
    print("\nConfusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

    return accuracy

# Test the model
test_accuracy = evaluate_model(model, test_loader, device)


NameError: name 'model' is not defined